In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.hub.get_dir()
# torch.hub.set_dir("/scratch/frankyu/torch/hub")
torch.hub.get_dir()

'/ubc/cs/home/f/frankyu/.cache/torch/hub'

In [3]:
 # Download model and configuration from S3 and cache.
config = torch.hub.load('huggingface/pytorch-transformers', 'config', 'DeepPavlov/bert-base-multilingual-cased-sentence')

Using cache found in /ubc/cs/home/f/frankyu/.cache/torch/hub/huggingface_pytorch-transformers_master


In [4]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'DeepPavlov/bert-base-multilingual-cased-sentence', config=config)

Using cache found in /ubc/cs/home/f/frankyu/.cache/torch/hub/huggingface_pytorch-transformers_master


In [6]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'DeepPavlov/bert-base-multilingual-cased-sentence')

Using cache found in /ubc/cs/home/f/frankyu/.cache/torch/hub/huggingface_pytorch-transformers_master


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

In [26]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [81]:
CosineSimilarity = nn.CosineSimilarity()

In [104]:
# test_sentence = "How late will the bar be open?"
test_sentence = "How late will the bar be open?"
test_sentence_german = "Wie spät wird die Bar geöffnet sein?"

input_german = tokenizer(test_sentence_german)
input = tokenizer(test_sentence)

german_output = model(torch.tensor(input_german['input_ids']).unsqueeze(0), torch.tensor(input_german['attention_mask']).unsqueeze(0))
english_output = model(torch.tensor(input['input_ids']).unsqueeze(0), torch.tensor(input['attention_mask']).unsqueeze(0))

print(german_output['pooler_output'].shape)
print(english_output['pooler_output'].shape)

distance = CosineSimilarity(english_output['pooler_output'], german_output['pooler_output'])
print(distance)

torch.Size([1, 768])
torch.Size([1, 768])
tensor([0.9801], grad_fn=<DivBackward0>)


In [105]:
test_sentence = "バーへの行き方を教えてください。"
test_sentence_german = "How do I get to the bar?"
# test_sentence_german = "mein Name ist Frank."

input_german = tokenizer(test_sentence_german)
input = tokenizer(test_sentence)

german_output = model(torch.tensor(input_german['input_ids']).unsqueeze(0), torch.tensor(input_german['attention_mask']).unsqueeze(0))
english_output = model(torch.tensor(input['input_ids']).unsqueeze(0), torch.tensor(input['attention_mask']).unsqueeze(0))

print(german_output['pooler_output'].shape)
print(english_output['pooler_output'].shape)

distance = CosineSimilarity(english_output['pooler_output'], german_output['pooler_output'])
print(distance)

torch.Size([1, 768])
torch.Size([1, 768])
tensor([0.9038], grad_fn=<DivBackward0>)


In [107]:
import pandas as pd
test_data = pd.read_csv ("data/XNLI-15way/xnli.15way.orig.tsv", sep = '\t')

In [146]:
# languages = ['fr', 'es', 'de', 'el', 'bg', 'ru', 'tr', 'ar', 'vi', 'th', 'zh', 'hi', 'sw', 'ur']
# languages = ['fr', 'es', 'de', 'el', 'bg', 'ru', 'tr', 'ar', 'vi', 'th', 'zh', 'hi']
languages = ['de']
print(len(languages))

1


In [147]:
num_examples = len(test_data[languages[0]])
print(num_examples)

10000


In [148]:
print(test_data[languages[0]][0])

und er hat gesagt, Mama ich bin daheim.


In [149]:
XNLI_similarity = []
for n in range(num_examples):
    if n % 100 == 0:
        print("{}/{}".format(n,num_examples))
    if n == 1000:
        break
    example_cos_sim = []
    eng_input = test_data['en'][n]
    eng_input = tokenizer(eng_input)
    english_output = model(torch.tensor(eng_input['input_ids']).unsqueeze(0), \
                           torch.tensor(eng_input['attention_mask']).unsqueeze(0))
    
    for l in languages:
        lang_input = test_data[l][n]
        lang_input = tokenizer(lang_input)
        lang_output = model(torch.tensor(lang_input['input_ids']).unsqueeze(0), \
                            torch.tensor(lang_input['attention_mask']).unsqueeze(0))
        distance = CosineSimilarity(english_output['pooler_output'].detach(), lang_output['pooler_output'].detach())
        example_cos_sim.append(distance)
        
    example_cos_sim = torch.stack(example_cos_sim)
    XNLI_similarity.append(torch.mean(example_cos_sim))

print(len(XNLI_similarity))
XNLI_similarity = torch.stack(XNLI_similarity)
overal_sim = torch.mean(XNLI_similarity)
print(overal_sim)

0/10000
100/10000
200/10000
300/10000
400/10000
500/10000
600/10000
700/10000
800/10000
900/10000
1000/10000
1000
tensor(0.9166)


In [136]:
old_model = torch.hub.load('huggingface/pytorch-transformers',\
                           'model', \
                           'bert-base-multilingual-cased')

Using cache found in /ubc/cs/home/f/frankyu/.cache/torch/hub/huggingface_pytorch-transformers_master


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [137]:
print(old_model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [150]:
# TEST W/ pretrained mBERT
XNLI_similarity = []
for n in range(num_examples):
    if n % 100 == 0:
        print("{}/{}".format(n,num_examples))
    if n == 1000:
        break
    example_cos_sim = []
    eng_input = test_data['en'][n]
    eng_input = tokenizer(eng_input)
    english_output = old_model(torch.tensor(eng_input['input_ids']).unsqueeze(0), \
                           torch.tensor(eng_input['attention_mask']).unsqueeze(0))
    
    for l in languages:
        lang_input = test_data[l][n]
        lang_input = tokenizer(lang_input)
        lang_output = old_model(torch.tensor(lang_input['input_ids']).unsqueeze(0), \
                            torch.tensor(lang_input['attention_mask']).unsqueeze(0))
        distance = CosineSimilarity(english_output['pooler_output'].detach(), lang_output['pooler_output'].detach())
        example_cos_sim.append(distance)
        
    example_cos_sim = torch.stack(example_cos_sim)
    XNLI_similarity.append(torch.mean(example_cos_sim))

print(len(XNLI_similarity))
XNLI_similarity = torch.stack(XNLI_similarity)
overal_sim = torch.mean(XNLI_similarity)
print(overal_sim)

0/10000
100/10000
200/10000
300/10000
400/10000
500/10000
600/10000
700/10000
800/10000
900/10000
1000/10000
1000
tensor(0.9191)


# Results from Preliminary Test

## Test Explaination:
In this test, I compare English embeddings to embeddings of different subset of the 14 others in XNLI dataset (with the cosine distance). Distances averages for multi-language settings are just compared between target language and english as the control (to save computation) and is only evaluated on the first 1000 sentences (to save time). The following table shows the performance of both mSBERT (fine-tuned to have more consistent embeddings across all 15 languages) and the pre-trained mBERT. 

|        | All Lang. | -sw -ur | german |
|--------|-----------|---------|--------|
| mSBERT | 0.8772    | 0.8864  | 0.9166 |
| mBERT  | 0.8233    | 0.8366  | 0.9191 |


## Conclusions:
Based on these preliminary experiments it seems like mSBERT is capable of better representing all the sentences while maintaining a very similar performance on the German-English test (which has been shown to be an "easier" tasks. 